# Model in-prod, debugging, run: "PTD-80"

In [ ]:
import pickle
import shap

import neptune.new as neptune
import pandas as pd
import xgboost as xgb

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [ ]:
rnd_state = 987245
base_namespace = "model_debugging"
run_id = "PTD-80"

## Fetch project and resume run

In [ ]:
project = neptune.get_project(name="common/project-tabular-data")

run = neptune.init(
    project="common/project-tabular-data",
    run=run_id,
    monitoring_namespace=f"{base_namespace}/monitoring",
)

## Download model from the run

In [ ]:
run["model_training/pickled_model"].download("xgb.model")
with open("xgb.model", "rb") as file:
    bst = pickle.load(file)

## Prepare data

In [ ]:
data = pd.read_csv('../data/train.csv')
data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = data.SalePrice
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

## Run model debugging logic and log results

### Look at specific trees

In [ ]:
ax = xgb.plot_tree(bst, num_trees=5)

In [ ]:
run[f"{base_namespace}/trees"].log(neptune.types.File.as_image(ax.figure))

In [ ]:
ax = xgb.plot_tree(bst, num_trees=6)

In [ ]:
run[f"{base_namespace}/trees"].log(neptune.types.File.as_image(ax.figure))

### SHAP explainers

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(bst)
shap_values = explainer.shap_values(X)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X)

In [ ]:
shap.summary_plot(shap_values, X)

## Mark run as "debugged" and close it

In [ ]:
run["sys/tags"].add("debugged")

In [ ]:
run.stop()